<a href="https://colab.research.google.com/github/WuzI38/Data/blob/Jupyter/MidiGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Second attempt on music generation

## Download data

In [2]:
!pip install -q kaggle

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [4]:
!mkdir -p ~/.kaggle
!cp /content/gdrive/MyDrive/Kaggle/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle datasets download programgeek01/anime-music-midi

anime-music-midi.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# Unzip data
import zipfile

zip_ref = zipfile.ZipFile("anime-music-midi.zip", "r") # Hey, don't blame me for the name, I am not this dataset's creator
zip_ref.extractall()
zip_ref.close()

In [7]:
# enables music21 to render images of musical notes
print('installing lilypond...')
!apt-get install lilypond > /dev/null

# enables playing midi files
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null

# Copy soundfont to content directory
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

installing lilypond...
installing fluidsynth...


## Loading and preprocessing

### Parsing files

In [ ]:
import os
from music21 import *
from fractions import Fraction

path = "/content/data/undertale/"
midi_list = []
for filename in os.listdir(path):
  # print(filename)
  parsed_file = converter.parse(path + filename)
  midi_list.append(parsed_file)

In [ ]:
midi_list[0]

### Extracting notes and chords



In [ ]:
import music21
def fragmentise(midi_stream: music21.stream.Score) -> list:
  return [part.flat.notes for part in midi_stream.parts]

In [ ]:
def signature(midi_stream: music21.stream.Score) -> list:
  sig = midi_stream.getTimeSignatures()[0]
  return [sig.numerator, sig.denominator]

In [ ]:
frag = fragmentise(midi_list[2])
frag

In [ ]:
sg = signature(frag[0])
sg

In [ ]:
# Part one - extract notes from both streams and save them into a list
def get_all_notes(midi_piece: music21.stream.Score) -> list:
  all_notes = list()
  note_types = set()
  midi_part = fragmentise(midi_piece)[0] # use only notes that belong to main melody
  for nt in midi_part.flat.notes: 
    if isinstance(nt, note.Note):
      pitch = str(max(0.0, nt.pitch.ps))
    elif isinstance(nt, chord.Chord):
      pitch = sorted([str(max(0.0, n.ps)) for n in nt.pitches])
      pitch = ' '.join(pitch)
    ql = nt.duration.quarterLength
    all_notes.append([nt.offset, pitch, ql if ql > 0.0 else 0.25]) # If for some reason len is 0 replace it with 0.25
  return all_notes
  
notes_list = get_all_notes(midi_list[1])
notes_list[:10]

In [ ]:
# Part 2 extract all chord types from the dataset
import numpy as np

def get_chord_types(dataset: list) -> dict:
  chord_types = dict()
  for d in dataset:
    notes_list = np.array(get_all_notes(d))
    strings = notes_list[:, 1]
    for s in strings:
      chord_types[s] = 1 + chord_types[s] if chord_types.get(s) is not None else 1
  return chord_types

types = get_chord_types(midi_list)

In [ ]:
print(len(types))

### Counting invidual chords and removing the rare ones

In [ ]:
chord_counts = dict()
for item in sorted(list(types.values())):
  chord_counts[str(item)] = 1 + chord_counts[str(item)] if chord_counts.get(str(item)) is not None else 1

chord_counts = {k: v for k, v in chord_counts.items() if v > 1}

In [ ]:
# Plotting the values
import matplotlib.pyplot as plt

# Extract the keys and values from the dictionary as lists
keys = list(chord_counts.keys())
values = list(chord_counts.values())

# Change size
plt.figure(figsize=(16, 12))

# Create a bar chart using the plot function
plt.bar(keys, values)

# Add a title and axis labels
plt.title('Number of types among chords with the same popularity (only values > 1)')
plt.xlabel('Popularity')
plt.ylabel('Types')

# Rotate labels
plt.xticks(rotation=90)

# Display the plot
plt.show()

In [ ]:
from statistics import mean
HOW_MANY = 20
means = [mean([len(k)//4 for k, v in types.items() if v <= i]) for i in range(1, HOW_MANY + 1)]

In [ ]:
# Plotting the means
plt.figure(figsize=(10, 7))

# Create a bar chart using the plot function
plt.bar(list(range(1, HOW_MANY + 1)), means)

# Add a title and axis labels
plt.title('Avg length of rare chords')
plt.xlabel('Popularity of a chord')
plt.ylabel('Avg length')

# Rotate labels
plt.xticks(rotation=90)

# Display the plot
plt.show()

The avg length is around 3 for the rare chords, so I can probably shorten them to 2 notes for the encoding purposes, so i don't have to use 1881 different values

## Visualize melodies

In [ ]:
import music21
from IPython.display import Image, Audio

def show(music):
  display(Image(str(music.write('lily.png'))))

def play(music):
    try:
        filename = music.write('mid')
        os.system(f'fluidsynth -ni font.sf2 {filename} -F {filename}.wav -r 16000 > /dev/null')
        display(Audio(f'{filename}.wav'))
    except Exception as e:
        print(f'Error: {e}')

In [ ]:
n = music21.note.Note('D#3')
n.duration = duration.Duration(Fraction(1, 3))
show(n)

In [ ]:
play(n)

In [ ]:
from music21 import chord
def create_midi_part(part_n: music21.stream.Score) -> list:
  melody = get_all_notes(part_n)
  melody_ready = []
  # Create really ugly and unnecessary for loop just for 
  for sound in melody:
    if len(sound[1]) <= 4:
      new_note = note.Note(int(float(sound[1])), quarterLength = sound[2]) # use .nameWithOctave for the names only
    else:
      pitches_str = sound[1].split()
      pitches = [int(float(s)) for s in pitches_str]
      new_note = chord.Chord(pitches, quarterLength = sound[2]) # Must be an int (despite the fact that GPT says otherwise)
    new_note.offset = sound[0]
    melody_ready.append(new_note)
  return melody_ready

In [ ]:
melody_visual = create_midi_part(midi_list[15])
melody_midi = stream.Stream(melody_visual)

In [ ]:
show(melody_midi[:50])

In [ ]:
play(melody_midi[:50])

## Simpifying chords and encoding

In [ ]:
LIMIT = 8
TRIM_TO = 9
TRIM_TO = TRIM_TO if (TRIM_TO - 4) % 5 == 0 else 9 # Choose correct trim

### Check if removing rare chords helps
ENCODER = dict() # checking if trimming chords makes any sense at all + creating id for each chord
TRIMMER = dict() # enables transformation of chords
id = 2
for key, value in types.items(): 
  kcp = key[:TRIM_TO] if value <= LIMIT and len(key) > TRIM_TO else key
  TRIMMER[key] = kcp
  if ENCODER.get(kcp) is None:
    ENCODER[kcp] = id
    id += 1

print(len(types), len(ENCODER))

### Encoding and decoding

In [ ]:
notes_list[:10]

In [ ]:
# Warning!!! This method removes rare chords, so the encoded version may not be the same
# Pause id -> pause, empty_id -> no new notes

def encode_part(notes: list, chord_trans: dict, chords_id: dict, empty_id: int = 1, pause_id: int = 0) -> list:
  sound_list = list()
  for index, value in enumerate(notes[:-1]):
    cnt = int(value[2] * 12)
    offest_dif = int(12 * (notes[index + 1][0] - value[0]))
    for x in range(cnt):
      new_symbol = chords_id[chord_trans[value[1]]] if not x else empty_id
      sound_list.append(new_symbol)
    for _ in range(max(offest_dif - cnt, 0)):
      sound_list.append(pause_id)
  cnt = int(notes[-1][2] * 12)
  for x in range(cnt):
      new_symbol = chords_id[chord_trans[notes[-1][1]]] if not x else empty_id
      sound_list.append(new_symbol)
  return sound_list

In [ ]:
part_encoded = encode_part(get_all_notes(midi_list[1]), TRIMMER, ENCODER)
part_encoded[:20]

In [ ]:
# Simply transform notes of len 1/3 into python fractions
def float_to_fraction(x: float):
    frac = Fraction(x).limit_denominator()
    return frac if (frac.denominator % 3 == 0 and frac.numerator % frac.denominator != 0) else x

print(float_to_fraction(0.25), float_to_fraction(0.3333333333333))

In [ ]:
def decode_part(encoded: list, chords_id_encoder: dict, empty_id: int = 1, pause_id: int = 0) -> list:
  notes_decoded = list()
  counter = 1
  zero_counter = 0
  offset = 0
  last_n = encoded[0]
  for n in encoded[1:]:
    if n not in (empty_id, pause_id):
      notes_decoded.append([offset, chords_id_encoder[last_n], float_to_fraction(counter/12)])
      offset += float((counter + zero_counter) / 12)
      counter = 1
      zero_counter = 0
      last_n = n
    elif n == empty_id:
      counter += 1
    elif n == pause_id:
      zero_counter += 1
  notes_decoded.append([offset, chords_id_encoder[last_n], float_to_fraction(counter/12)])
      
  return notes_decoded

In [ ]:
DECODER = {v: k for k, v in ENCODER.items()}
lst = decode_part(part_encoded, DECODER)
lst[:10]

Warning: there is a certain issue if it comes to midi files. Sometimes 2 separate notes which do not belong to a chord have the same offsets - in that case the offset of the second note is shifted by its length

## Windowing + one hot encoding
As chords and notes are discrete values, one hot encoding might be a good idea ???
As octaves are represented in my encoding as 48 values (4 quarter notes are represented by 12 digits) the horizon value will be 

### Windowing

In [ ]:
# As 
HORIZON = 12
WINDOW_SIZE = 48

In [ ]:
# Create function to label windowed data
def get_labelled_windows(x: np.ndarray, horizon=12):
  return x[:, :-horizon], x[:, -horizon:]

In [ ]:
def make_windows(x: np.ndarray, window_size: int = WINDOW_SIZE, horizon: int = HORIZON) -> np.ndarray:
  # 1. Create a window of specific window size (add the horizon on the end for labelling later)
  window_step = np.expand_dims(np.arange(window_size + horizon), axis=0)

  # 2. Use numpy indexing to create a 2D array of multiple windows
  window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).transpose()
  # len(x)-(window_size+horizon-1) is used to prevent sliding window from getting out of range
  # Basically for some reason the sum of [[0, 1, 2]] and [[0], [1], [2]] gives you [[0, 1, 2], [1, 2, 3], [2, 3, 4]]

  # 3. Index on the target array (a time series) with 2D array of multiple window steps
  windowed_array = x[window_indexes]

  # 4. Get the labelled windows
  windows, labels = get_labelled_windows(windowed_array, horizon=horizon)

  return windows, labels

In [ ]:
test_windows, test_labels = make_windows(np.array(part_encoded), window_size=WINDOW_SIZE, horizon=HORIZON)

In [ ]:
test_windows[:5]

In [ ]:
test_labels[:5]

### Joining, one hot encoding the dataset and creating train/test split

### Joining and creating splits

The function removes pieces shorter than 40 notes by default - I assume there are not very important for the whole dataset. The function does not work properly if the piece given as an argument is shorter than 5 quarter notes

In [ ]:
TRAIN_TEST = 0.8

def join_midis(midi_list: list, 
               trimmer: dict, 
               encoder: dict, 
               train_test: np.float128 = TRAIN_TEST,
               empty_id: int = 1, 
               pause_id: int = 0,
               window_size: int = WINDOW_SIZE, 
               horizon: int = HORIZON,
               shortest_piece: int = 40) -> np.ndarray:
  
  X_train = None
  y_train = None
  X_test = None
  y_test = None
  
  for midi in midi_list:
    notes = get_all_notes(midi)
    if len(notes) < shortest_piece:
      continue
    divider = int(TRAIN_TEST * len(notes))
    train, test = notes[:divider], notes[divider:]

    train_part_encoded = encode_part(train, trimmer, encoder)
    test_part_encoded = encode_part(test, trimmer, encoder)

    train_windows, train_labels = make_windows(np.array(train_part_encoded), window_size, horizon)
    test_windows, test_labels = make_windows(np.array(test_part_encoded), window_size, horizon)

    if X_train is None:
      X_train = train_windows
      y_train = train_labels
      X_test = test_windows
      y_test = test_labels
    else:
      X_train = np.concatenate([X_train, train_windows])
      y_train = np.concatenate([y_train, train_labels])
      X_test = np.concatenate([X_test, test_windows])
      y_test = np.concatenate([y_test, test_labels])

  return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = join_midis(midi_list, TRIMMER, ENCODER)

### One hot encoding

In [ ]:
from tensorflow.keras.utils import to_categorical